# Install Packages

In [ ]:
!pip3 install wikipedia-api


In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=3f71bef4edc3c60c0125f427787346fb4a74ae08741597027952e6a895755e02
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
!pip install requests beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_wikipedia_page(query):
    """
    获取与查询最相关的 Wikipedia 页面内容
    """
    # 搜索 Wikipedia API
    search_url = "https://en.wikipedia.org/w/api.php"
    search_params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json",
    }

    response = requests.get(search_url, params=search_params)
    if response.status_code != 200:
        print("搜索请求失败！")
        return "","",""

    search_results = response.json()
    if not search_results["query"]["search"]:
        print("没有找到相关结果！")
        return "","",""

    # 获取第一个结果的标题
    page_title = search_results["query"]["search"][0]["title"]


    # 获取页面内容
    page_url = f"https://en.wikipedia.org/wiki/{page_title.replace(' ', '_')}"
    page_response = requests.get(page_url)
    if page_response.status_code != 200:
        print("获取页面内容失败！")
        return "","",""

    soup = BeautifulSoup(page_response.text, "html.parser")
    content = soup.find("div", {"id": "mw-content-text"})
    if not content:
        print("未找到页面内容！")
        return "","",""

    # 提取纯文本内容
    paragraphs = content.find_all("p")
    page_content = "\n".join([p.get_text() for p in paragraphs if p.get_text().strip()])

    return page_title, page_url, page_content


if __name__ == "__main__":
    query = input("请输入查询词: ")
    result = get_wikipedia_page(query)
    if result:
        title, url, content = result
        print(f"\n页面标题: {title}")
        print(f"页面链接: {url}")
        print(f"\n页面内容预览:\n{content[:1000]}...")  # 只显示前1000个字符


请输入查询词: trump

页面标题: Trump
页面链接: https://en.wikipedia.org/wiki/Trump

页面内容预览:
Trump most commonly refers to:

Trump may also refer to:
...


# DataSet

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

nq_ds = load_dataset("sentence-transformers/natural-questions")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/44.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100231 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
nq_train_df = pd.DataFrame(nq_ds['train'])

# Pure LLM

In [ ]:
from huggingface_hub import InferenceClient

def llama_answer(
    question: str, model_name="meta-llama/Meta-Llama-3-8B-Instruct", external_knowledge=None
) -> str:

    prompt = "You are an expert answering common sense question.\n"
    if external_knowledge:
        prompt += f"You may find this information useful: {external_knowledge}.\n"
    prompt += f"Question: {question}\n"
    client = InferenceClient(api_key="YOUR_API_KEY")

    messages = [{"role": "user", "content": prompt}]

    stream = client.chat.completions.create(
        model=model_name, messages=messages, max_tokens=5000, stream=True
    )
    # Initialize an empty string to store the accumulated result
    result_string = ""

    # Iterate through the stream and accumulate content into the result string
    for chunk in stream:
        result_string += chunk.choices[0].delta.content
    return result_string


def gpt_answer(question: str, model_name="gpt-4o-mini",external_knowledge=None) -> str:
    client = OpenAI()
    prompt = f"You are an expert at common sense.\n"
    if external_knowledge:
        prompt += f"You may find this information useful: {external_knowledge}\n"
    messages = [
        {"role": "system", "content":prompt},
        {"role": "user", "content": question},
    ]
    try:
        response = client.chat.completions.create(model=model_name, messages=messages)
        output = response.choices[0].message.content.strip()
        return output
    except Exception as e:
        print(f"Error: {e}")
        return None


# Navie RAG


In [ ]:
from openai import OpenAI
import os

client=OpenAI()

def extract_title(question: str) -> str:
    """
    Extract key words from a given question using OpenAI GPT or similar NLP techniques.

    :param question: The input question as a string.
    :return: A list of extracted keywords.
    """
    prompt = f"""
        You are a knowledgeable and concise assistant. I will provide you with a question,
        and your task is to identify the most relevant Wikipedia article title where I can
        find a correct and comprehensive answer to this question.

        Please provide your response as a single title.

        Here is the question:
        "{question}"
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        output = response.choices[0].message.content.strip()
        # Split the response into a list of keywords
        return output
    except Exception as e:
        print(f"Error: {e}")
        return ""





In [ ]:
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = None  # 如果不需要 OpenAI 模型，显式禁用
def chunk_text(text, chunk_size, chunk_overlap):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks

def embed_text(long_text, chunk_size=100, chunk_overlap=10, top_k=10):
    text_chunks = chunk_text(long_text, chunk_size, chunk_overlap)

    chunked_documents = [Document(text=chunk) for chunk in text_chunks]

    index = VectorStoreIndex.from_documents(chunked_documents)
    query_engine = index.as_query_engine(similarity_top_k=top_k)

    return query_engine

# 示例：嵌入一个非常长的字符串
long_text = """The University of California, Berkeley (UC Berkeley, Berkeley, Cal, or California)[10][11] is a public land-grant research university in Berkeley, California, United States. Founded in 1868 and named after the Anglo-Irish philosopher George Berkeley, it is the state's first land-grant university and is the founding campus of the University of California system.[12]

Berkeley has an enrollment of more than 45,000 students. The university is organized around fifteen schools of study on the same campus, including the College of Chemistry, the College of Engineering, and the Haas School of Business. It is classified among "R1: Doctoral Universities – Very high research activity".[13] The Lawrence Berkeley National Laboratory was originally founded as part of the university.[14]

Berkeley was a founding member of the Association of American Universities and was one of the original eight "Public Ivy" schools. In 2021, the federal funding for campus research and development exceeded $1 billion.[15] Thirty-two libraries also compose the Berkeley library system which is the sixth largest research library by number of volumes held in the United States.[16][17][18]

Berkeley students compete in thirty varsity athletic sports, and the university is one of eighteen full-member institutions in the Atlantic Coast Conference (ACC). Berkeley's athletic teams, the California Golden Bears, have also won 107 national championships, 196 individual national titles, and 223 Olympic medals (including 121 gold).[19][20] Berkeley's alumni, faculty, and researchers include 59 Nobel laureates[21] and 19 Academy Award winners,[22] and the university is also a producer of Rhodes Scholars,[23] Marshall Scholars,[24] and Fulbright Scholars.[25]

History

View from Memorial Glade of Sather Tower (the Campanile), the center of Berkeley

Sather Tower (the Campanile) looking out over San Francisco Bay and Mount Tamalpais
Main article: History of the University of California, Berkeley
Founding
Made possible by President Lincoln's signing of the Morrill Act in 1862, the University of California was founded in 1868 as the state's first land-grant university, inheriting the land and facilities of the private College of California and the federal-funding eligibility of a public agricultural, mining, and mechanical arts college.[26] The Organic Act states that the "University shall have for its design, to provide instruction and thorough and complete education in all departments of science, literature and art, industrial and professional pursuits, and general education, and also special courses of instruction in preparation for the professions."[27][28]

Ten faculty members and forty male students made up the fledgling university when it opened in Oakland in 1869.[29] Frederick Billings, a trustee of the College of California, suggested that a new campus site north of Oakland be named in honor of Anglo-Irish philosopher George Berkeley.[30] The university began admitting women the following year.[31] In 1870, Henry Durant, founder of the College of California, became its first president. With the completion of North and South Halls in 1873, the university relocated to its Berkeley location with 167 male and 22 female students.[32][33] The first female student to graduate was in 1874, admitted in the first class to include women in 1870.[34]

Beginning in 1891, Phoebe Apperson Hearst funded several programs and new buildings and, in 1898, sponsored an international competition in Antwerp, where French architect Émile Bénard submitted the winning design for a campus master plan. Although the University of California system does not have an official flagship campus, many scholars and experts consider Berkeley to be its unofficial flagship. It shares this unofficial status with the University of California, Los Angeles.[35][36]

20th century
In 1905, the University Farm was established near Sacramento, ultimately becoming the University of California, Davis.[37] In 1919, the Los Angeles branch of the California State Normal School became the southern branch of the university, which ultimately became the University of California, Los Angeles.[38] By the 1920s, the number of campus buildings in Berkeley had grown substantially and included twenty structures designed by architect John Galen Howard.[39] In 1917, one of the nation's first ROTC programs was established at Berkeley[40] and its School of Military Aeronautics began training pilots, including Jimmy Doolittle. In 1926, future Fleet Admiral Chester W. Nimitz established the first Naval ROTC unit at Berkeley.[41] Berkeley ROTC alumni include former Secretary of Defense Robert McNamara, Army Chief of Staff Frederick C. Weyand, sixteen other general officers, ten Navy flag officers, and AFROTC alumna Captain Theresa Claiborne.[42]

In the 1930s, Ernest Orlando Lawrence helped establish the Radiation Laboratory (now Lawrence Berkeley National Laboratory) and invented the cyclotron, which won him the Nobel physics prize in 1939.[43] Using the cyclotron, Berkeley professors and Berkeley Lab researchers went on to discover sixteen chemical elements—more than any other university in the world.[44][45] In particular, during World War II and following Glenn Seaborg's then-secret discovery of plutonium, Lawrence's Radiation Laboratory began to contract with the U.S. Army to develop the atomic bomb. Physics professor J. Robert Oppenheimer was named scientific head of the Manhattan Project in 1942.[46][47] Along with the Lawrence Berkeley National Laboratory, Berkeley founded and was then a partner in managing two other labs, Los Alamos National Laboratory (1943) and Lawrence Livermore National Laboratory (1952).

The "Bodies Upon the Gears" speech (also known as "Operation of the Machine") given by Mario Savio on the steps of Sproul Hall in 1964
In 1952, the University of California reorganized itself into a system of semi-autonomous campuses, with each campus given a chancellor, and Clark Kerr became Berkeley's first Chancellor, while  Robert Sproul remained in place as the President of the University of California.[48] Berkeley gained a worldwide reputation for political activism in the 1960s. In 1964, the Free Speech Movement organized student resistance to the university's restrictions on political activities on campus—most conspicuously, student activities related to the Civil Rights Movement.[49][50]

The arrest in Sproul Plaza of Jack Weinberg, a recent Berkeley alumnus and chair of Campus CORE, prompted a series of student-led acts of formal remonstrance and civil disobedience that ultimately gave rise to the Free Speech Movement, which movement would prevail and serve as a precedent for student opposition to America's involvement in the Vietnam War.[51][52][53] In 1982, the Mathematical Sciences Research Institute (MSRI) was established on campus with support from the National Science Foundation and at the request of three Berkeley mathematicians—Shiing-Shen Chern, Calvin Moore, and Isadore M. Singer. The institute is now widely regarded as a leading center for collaborative mathematical research, drawing thousands of visiting researchers from around the world each year.[54][55][56]

21st century
In the current century, Berkeley has become less politically active, although more liberal.[57][58] Democrats outnumber Republicans on the faculty by a ratio of nine to one, which is a ratio similar to that of American academia generally.[59] The school has become more focused on STEM disciplines and fundraising.[60][61][62] In 2007, the Energy Biosciences Institute was established with funding from BP and Stanley Hall, a research facility and headquarters for the California Institute for Quantitative Biosciences, opened. Supported by a grant from alumnus James Simons, the Simons Institute for the Theory of Computing was established in 2012. In 2015, Berkeley and its sister campus, UCSF, established the Innovative Genomics Institute to develop CRISPR gene editing, and, in 2020, an anonymous donor pledged $252 million to help fund a new center for computing and data science. For the 2020 fiscal year, Berkeley set a fundraising record, receiving over $1 billion in gifts and pledges, and two years later, it broke that record, raising over $1.2 billion.[63][60][64][65]

Controversies
Various research ethics, human rights, and animal rights advocates have been in conflict with Berkeley. Native Americans contended with the school over repatriation of remains from the Phoebe A. Hearst Museum of Anthropology.[66] Student activists have urged the university to cut financial ties with Tyson Foods and PepsiCo.[67][68][69] Faculty member Ignacio Chapela prominently criticized the university's financial ties to Novartis.[70] PETA has challenged the university's use of animals for research and argued that it may violate the Animal Welfare Act.[71][72]
Cal's Memorial Stadium reopened in September 2012 after renovations. The university incurred a controversial $445 million of debt for the stadium and a new $153 million student athletic center, which it financed with the sale of special stadium endowment seats.[73] The roughly $18 million interest-only annual payments on the debt consumes 20 percent of Cal's athletics' budget; principal repayment begins in 2032 and is scheduled to conclude in 2113.[74]
On May 1, 2014, Berkeley was named one of fifty-five higher education institutions under investigation by the U.S. Department of Education's Office of Civil Rights "for possible violations of federal law over the handling of sexual violence and harassment complaints" by the White House Task Force to Protect Students from Sexual Assault.[75] Investigations continued into 2016, with hundreds of pages of records released in April 2016, showing a pattern of documented sexual harassment and firings of non-tenured staff.[76]
On July 25, 2019, Berkeley was removed from the U.S. News Best Colleges Ranking for misreporting statistics. Berkeley had originally reported that its two-year average alumni giving rate for fiscal years 2017 and 2016 was 11.6 percent, U.S. News said. The school later told U.S. News the correct average alumni giving rate for the 2016 fiscal year was just 7.9 percent. The school incorrectly overstated its alumni giving data to U.S. News since at least 2014. The alumni giving rate accounts for five percent of the Best Colleges ranking.[77]
Berkeley community members have criticized UC Berkeley's increasing enrollment. Berkeley residents filed a lawsuit alleging that the university's expanding enrollment violated California Environmental Quality Act and that the area lacked the infrastructure to support more students.[78] Critics
"""

# 嵌入并创建查询引擎
query_engine = embed_text(long_text)

# 示例查询
query_result = query_engine.query("U.S. News")
print("查询结果:", query_result)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.
查询结果: Context information is below.
---------------------
he school incorrectly overstated its alumni giving data to U.S. News since at least 2014. The alumni

fiscal years 2017 and 2016 was 11.6 percent, U.S. News said. The school later told U.S. News the cor

2019, Berkeley was removed from the U.S. News Best Colleges Ranking for misreporting statistics. Be

e sixth largest research library by number of volumes held in the United States.[16][17][18]

Berkel

er investigation by the U.S. Department of Education's Office of Civil Rights "for possible violatio

ctions on political activities on campus—most conspicuously, student activities related to the Civil

raising over $1.2 billion.[63][60][64][65]

Controversies
Various research ethics, human rights, and

rld each year.[54][55][56]

21st century
In the current century, Berkeley has become less politicall

t, which movement would prevail and serve as a precedent for student opposition to A

In [ ]:
response = query_engine.query("U.S. News")
print("Final Answer:", response.response)

# 查看所有被检索到的文本块
for i, node_with_score in enumerate(response.source_nodes):
    print("------")
    print(f"Chunk #{i+1} (Similarity Score: {node_with_score.score}):")
    print(node_with_score.node.text)
    print("------")


Final Answer: Context information is below.
---------------------
he school incorrectly overstated its alumni giving data to U.S. News since at least 2014. The alumni

fiscal years 2017 and 2016 was 11.6 percent, U.S. News said. The school later told U.S. News the cor

2019, Berkeley was removed from the U.S. News Best Colleges Ranking for misreporting statistics. Be

e sixth largest research library by number of volumes held in the United States.[16][17][18]

Berkel

er investigation by the U.S. Department of Education's Office of Civil Rights "for possible violatio

ctions on political activities on campus—most conspicuously, student activities related to the Civil

raising over $1.2 billion.[63][60][64][65]

Controversies
Various research ethics, human rights, and

rld each year.[54][55][56]

21st century
In the current century, Berkeley has become less politicall

t, which movement would prevail and serve as a precedent for student opposition to America's involve

's Radiation Labo

In [ ]:
import pandas as pd
nq_train_df=pd.read_csv("llama_naive_evaluated.csv")

In [ ]:
import pandas as pd
from tqdm import tqdm

# 设置起始行
start_index = 400
N = 500

for index, row in tqdm(nq_train_df.iloc[start_index:N].iterrows(), total=N-start_index, desc="Processing rows"):
    query = row['query']
    answer = row['answer']
    print(f"Row {index}: Query={query}, Answer={answer}")

    naive_answer = llama_answer(query)

    key_words = extract_title(query)
    if key_words:
        url, title, content = get_wikipedia_page(key_words)
        query_engine = embed_text(content[:10000])
        query_result = query_engine.query(query)
        naive_rag_answer = llama_answer(question=query, external_knowledge=query_result.response)
    else:
        naive_rag_answer = None

    nq_train_df.at[index, 'naive_answer'] = naive_answer
    nq_train_df.at[index, 'naive_rag_answer'] = naive_rag_answer

nq_train_df.to_csv('nq_train_df_updated.csv', index=False)


Processing rows:   0%|          | 0/100 [00:00<?, ?it/s]

Row 400: Query=when did the blues last win super rugby, Answer=Blues (Super Rugby) The Blues (formerly known as the Auckland Blues until 2000) are a professional rugby union team based in Auckland, New Zealand who play in the Super Rugby competition. Like New Zealand's four other Super Rugby regional franchises, the Blues were established by the NZRU in 1996. One of the most successful teams in Super Rugby history, the Blues won the competition in each of its first two seasons, 1996 and 1997, and again in 2003. Additionally, the team were finalists in 1998 and semi-finalists in 2007 and 2011. The team is captained by James Parsons and coached by Tana Umaga.


Processing rows:   1%|          | 1/100 [00:05<09:40,  5.86s/it]

Row 401: Query=how many nba games in a season per team, Answer=National Basketball Association During the regular season, each team plays 82 games, 41 each home and away. A team faces opponents in its own division four times a year (16 games). Each team plays six of the teams from the other two divisions in its conference four times (24 games), and the remaining four teams three times (12 games). Finally, each team plays all the teams in the other conference twice apiece (30 games). This asymmetrical structure means the strength of schedule will vary between teams (but not as significantly as the NFL or MLB). Over five seasons, each team will have played 80 games against their division (20 games against each opponent, 10 at home, 10 on the road), 180 games against the rest of their conference (18 games against each opponent, 9 at home, 9 on the road), and 150 games against the other conference (10 games against each team, 5 at home, 5 on the road).


Processing rows:   2%|▏         | 2/100 [00:16<14:17,  8.75s/it]

...

Processing rows:  98%|█████████▊| 98/100 [15:46<00:19,  9.78s/it]

Row 498: Query=how did the shark die in jaws the book, Answer=Jaws (novel) When they return to sea the following day, the shark starts ramming the boat. Quint is able to harpoon it several times. The shark leaps out of the water and onto the stern of the Orca, ripping a huge hole in the aft section and causing the boat to start sinking. Quint plunges another harpoon into the shark's belly, but as it falls back into the water, his foot gets entangled in the rope, and he is dragged underwater to his death. Brody, now floating on a seat cushion, spots the shark swimming towards him and prepares for his death. However, just as the shark gets within a few feet of him, it succumbs to its many wounds, rolls over in the water and dies before it can kill Brody. The great fish sinks down out of sight, dragging Quint's still entangled body behind it. The lone survivor of the ordeal, Brody paddles back to shore on his makeshift float.


Processing rows:  99%|█████████▉| 99/100 [16:00<00:11, 11.16s/it]

Row 499: Query=when does braxton family values season 5 start, Answer=Braxton Family Values (season 5) The fifth season of Braxton Family Values, an American reality television series, broadcast on WE tv. It premiered on May 19, 2016, and was primarily filmed in Atlanta, Georgia and Los Angeles, California. Its executive producers are Toni Braxton, Tamar Braxton, Vincent Herbert, Dan Cutforth, Jane Lipsitz, Julio Kollerbohm, Michelle Kongkasuwan, Lauren Gellert, Annabelle McDonald and Sitarah Pendelton.


Processing rows: 100%|██████████| 100/100 [16:05<00:00,  9.65s/it]


In [ ]:
import pandas as pd
from tqdm import tqdm

# 假设 nq_train_df 已经被初始化并加载数据
N = 400

# 创建新的列以存储答案
nq_train_df['naive_answer'] = None
nq_train_df['naive_rag_answer'] = None

# for index, row in nq_train_df.head(N).iterrows():
for index, row in tqdm(nq_train_df.head(N).iterrows(), total=N, desc="Processing rows"):
    query = row['query']
    answer = row['answer']
    print(f"Row {index}: Query={query}, Answer={answer}")

    # 调用 llama_answer 函数获取 naive_answer
    naive_answer = llama_answer(query)

    # 提取关键词并进行 RAG 策略
    key_words = extract_title(query)
    if key_words:
        url, title, content = get_wikipedia_page(key_words)
        query_engine = embed_text(content[:10000])
        query_result = query_engine.query(query)
        naive_rag_answer = llama_answer(question=query, external_knowledge=query_result.response)
    else:
        naive_rag_answer = None

    # 更新 DataFrame
    nq_train_df.at[index, 'naive_answer'] = naive_answer
    nq_train_df.at[index, 'naive_rag_answer'] = naive_rag_answer

# 保存更新后的 DataFrame 为 CSV
nq_train_df.to_csv('nq_train_df_updated.csv', index=False)


Processing rows:   0%|          | 0/400 [00:00<?, ?it/s]

Row 0: Query=when did richmond last play in a preliminary final, Answer=Richmond Football Club Richmond began 2017 with 5 straight wins, a feat it had not achieved since 1995. A series of close losses hampered the Tigers throughout the middle of the season, including a 5-point loss to the Western Bulldogs, 2-point loss to Fremantle, and a 3-point loss to the Giants. Richmond ended the season strongly with convincing victories over Fremantle and St Kilda in the final two rounds, elevating the club to 3rd on the ladder. Richmond's first final of the season against the Cats at the MCG attracted a record qualifying final crowd of 95,028; the Tigers won by 51 points. Having advanced to the first preliminary finals for the first time since 2001, Richmond defeated Greater Western Sydney by 36 points in front of a crowd of 94,258 to progress to the Grand Final against Adelaide, their first Grand Final appearance since 1982. The attendance was 100,021, the largest crowd to a grand final since 1

Processing rows:   0%|          | 1/400 [00:01<12:07,  1.82s/it]

Row 1: Query=who sang what in the world's come over you, Answer=Jack Scott (singer) At the beginning of 1960, Scott again changed record labels, this time to Top Rank Records.[1] He then recorded four Billboard Hot 100 hits – "What in the World's Come Over You" (#5), "Burning Bridges" (#3) b/w "Oh Little One" (#34), and "It Only Happened Yesterday" (#38).[1] "What in the World's Come Over You" was Scott's second gold disc winner.[6] Scott continued to record and perform during the 1960s and 1970s.[1] His song "You're Just Gettin' Better" reached the country charts in 1974.[1] In May 1977, Scott recorded a Peel session for BBC Radio 1 disc jockey, John Peel.


Processing rows:   0%|          | 2/400 [00:02<07:47,  1.18s/it]

...

Processing rows: 100%|█████████▉| 398/400 [45:35<00:10,  5.22s/it]

Row 398: Query=who is catch me if you can based on, Answer=Catch Me If You Can Catch Me If You Can is a 2002 American biographical crime film directed and produced by Steven Spielberg from a screenplay by Jeff Nathanson. The film is based on the life of Frank Abagnale, who, before his 19th birthday, successfully performed cons worth millions of dollars by posing as a Pan American World Airways pilot, a Georgia doctor and a Louisiana parish prosecutor. His primary crime was check fraud; he became so experienced that the FBI eventually turned to him for help in catching other checking forgers. The film stars Leonardo DiCaprio and Tom Hanks, with Christopher Walken, Martin Sheen, and Nathalie Baye in supporting roles.


Processing rows: 100%|█████████▉| 399/400 [45:39<00:04,  4.82s/it]

Row 399: Query=when was the name burma changed to myanmar, Answer=Names of Myanmar Ma Thanegi records that the first use of the name 'Mranma' for the country is to be found on a 3 feet (91 cm) high stone inscription dated 597 ME (Traditional Burmese calendar) or 1235 CE. The stone is from the reign of Kyaswa, (1234-1250) son of King Htilominlo (Nadaungmya), Bagan. It is written in early Burmese script. Although the middle of the front side of this stone is damaged, the first line of the better-protected reverse side clearly shows မြန်မာပြည် ("Mranma kingdom"). The inscription is known as the 'Yadana Kon Htan Inscription'. At present it is in Bagan recorded as stone number 43 in the Archaeological Department's collection.[8]


Processing rows: 100%|██████████| 400/400 [45:50<00:00,  6.88s/it]


In [ ]:
import pandas as pd
from tqdm import tqdm

# 假设 nq_train_df 已经被初始化并加载数据
N = 500

# 创建新的列以存储答案
nq_train_df['naive_answer'] = None
nq_train_df['naive_rag_answer'] = None

# for index, row in nq_train_df.head(N).iterrows():
for index, row in tqdm(nq_train_df.head(N).iterrows(), total=N, desc="Processing rows"):
    query = row['query']
    answer = row['answer']
    print(f"Row {index}: Query={query}, Answer={answer}")

    # 调用 gpt_answer 函数获取 naive_answer
    naive_answer = gpt_answer(query)

    # 提取关键词并进行 RAG 策略
    key_words = extract_title(query)
    if key_words:
        url, title, content = get_wikipedia_page(key_words)
        query_engine = embed_text(content[:10000])
        query_result = query_engine.query(query)
        naive_rag_answer = gpt_answer(question=query, external_knowledge=query_result.response)
    else:
        naive_rag_answer = None

    nq_train_df.at[index, 'naive_answer'] = naive_answer
    nq_train_df.at[index, 'naive_rag_answer'] = naive_rag_answer

nq_train_df.to_csv('gpt_naive_rag.csv', index=False)

Processing rows:   0%|          | 0/500 [00:00<?, ?it/s]

Row 0: Query=when did richmond last play in a preliminary final, Answer=Richmond Football Club Richmond began 2017 with 5 straight wins, a feat it had not achieved since 1995. A series of close losses hampered the Tigers throughout the middle of the season, including a 5-point loss to the Western Bulldogs, 2-point loss to Fremantle, and a 3-point loss to the Giants. Richmond ended the season strongly with convincing victories over Fremantle and St Kilda in the final two rounds, elevating the club to 3rd on the ladder. Richmond's first final of the season against the Cats at the MCG attracted a record qualifying final crowd of 95,028; the Tigers won by 51 points. Having advanced to the first preliminary finals for the first time since 2001, Richmond defeated Greater Western Sydney by 36 points in front of a crowd of 94,258 to progress to the Grand Final against Adelaide, their first Grand Final appearance since 1982. The attendance was 100,021, the largest crowd to a grand final since 1

Processing rows:   0%|          | 1/500 [00:06<51:59,  6.25s/it]

Row 1: Query=who sang what in the world's come over you, Answer=Jack Scott (singer) At the beginning of 1960, Scott again changed record labels, this time to Top Rank Records.[1] He then recorded four Billboard Hot 100 hits – "What in the World's Come Over You" (#5), "Burning Bridges" (#3) b/w "Oh Little One" (#34), and "It Only Happened Yesterday" (#38).[1] "What in the World's Come Over You" was Scott's second gold disc winner.[6] Scott continued to record and perform during the 1960s and 1970s.[1] His song "You're Just Gettin' Better" reached the country charts in 1974.[1] In May 1977, Scott recorded a Peel session for BBC Radio 1 disc jockey, John Peel.


Processing rows:   0%|          | 2/500 [00:08<34:28,  4.15s/it]

...

Processing rows: 100%|█████████▉| 498/500 [44:10<00:12,  6.26s/it]

Row 498: Query=how did the shark die in jaws the book, Answer=Jaws (novel) When they return to sea the following day, the shark starts ramming the boat. Quint is able to harpoon it several times. The shark leaps out of the water and onto the stern of the Orca, ripping a huge hole in the aft section and causing the boat to start sinking. Quint plunges another harpoon into the shark's belly, but as it falls back into the water, his foot gets entangled in the rope, and he is dragged underwater to his death. Brody, now floating on a seat cushion, spots the shark swimming towards him and prepares for his death. However, just as the shark gets within a few feet of him, it succumbs to its many wounds, rolls over in the water and dies before it can kill Brody. The great fish sinks down out of sight, dragging Quint's still entangled body behind it. The lone survivor of the ordeal, Brody paddles back to shore on his makeshift float.


Processing rows: 100%|█████████▉| 499/500 [44:15<00:05,  5.89s/it]

Row 499: Query=when does braxton family values season 5 start, Answer=Braxton Family Values (season 5) The fifth season of Braxton Family Values, an American reality television series, broadcast on WE tv. It premiered on May 19, 2016, and was primarily filmed in Atlanta, Georgia and Los Angeles, California. Its executive producers are Toni Braxton, Tamar Braxton, Vincent Herbert, Dan Cutforth, Jane Lipsitz, Julio Kollerbohm, Michelle Kongkasuwan, Lauren Gellert, Annabelle McDonald and Sitarah Pendelton.


Processing rows: 100%|██████████| 500/500 [44:19<00:00,  5.32s/it]


In [ ]:
nq_train_df.head(3)

,query,answer,improved_rag_answer,naive_answer,naive_rag_answer
0,when did richmond last play in a preliminary f...,Richmond Football Club Richmond began 2017 wit...,NaN,"As of my last update in October 2023, the Rich...","Based on the context provided, there is no spe..."
1,who sang what in the world's come over you,"Jack Scott (singer) At the beginning of 1960, ...",NaN,"The song ""What in the World’s Come Over You"" w...","The song ""What's Come Over You"" is sung by Can..."
2,who produces the most wool in the world,Wool Global wool production is about 2 million...,NaN,As of my last knowledge update in October 2023...,The context provided does not specify which co...


In [ ]:
naive_answer

"The Richmond Football Club!\n\nAccording to the AFL records, Richmond's last appearance in a Preliminary Final was in 2001. They lost to the Brisbane Lions at the Melbourne Cricket Ground (MCG) by 43 points."

In [ ]:
navie_rag_answer

'Based on the provided context information, I can answer your query.\n\nThe text doesn\'t explicitly mention when Richmond last played in a preliminary final. The context only discusses Richmond\'s early years, their VFA and VFL history, and some notable events and matches. There is no mention of a specific preliminary final or its corresponding year.\n\nTherefore, I\'m unable to provide an answer to the question "when did Richmond last play in a preliminary final" as the required information is not present in the provided context. If you have additional context or information, I\'d be happy to try and help you answer the question.'

# Our Method


In [ ]:
import pandas as pd
import numpy as np
import os
from typing import List
import requests
from openai import OpenAI
from huggingface_hub import InferenceClient
def feedback_evaluate(query: str, retrieved_content: str, model_name="meta-llama/Meta-Llama-3-8B-Instruct") -> str:
    """
    Evaluate the relevance of retrieved content for a given query using Llama.

    :param query: The original query.
    :param retrieved_content: The retrieved content to be evaluated.
    :param model_name: The Llama model name.
    :return: Feedback as "Irrelevant", "Partially Relevant", or "Fully Relevant".
    """
    prompt = f"""
    You are an expert evaluator. Your task is to assess how relevant a piece of retrieved content is to a given query.
    The retrieved content should provide useful information. Don't be too strict.

    Here are the evaluation criteria:
    - "Fully Relevant": The content completely addresses the query with accurate and detailed information.
    - "Partially Relevant": The content somewhat addresses the query but lacks important details or includes unrelated information.
    - "Irrelevant": The content does not address the query or is unrelated to it.

    Query: {query}
    Retrieved Content: {retrieved_content}

    Based on the criteria above, return one of the following labels: "Fully Relevant", "Partially Relevant", or "Irrelevant".
    Don't return any other words.
    """

    client = InferenceClient(api_key="YOUR_API_KEY")

    messages = [{"role": "user", "content": prompt}]

    stream = client.chat.completions.create(
        model=model_name, messages=messages, max_tokens=100, stream=True
    )

    # Initialize an empty string to store the accumulated result
    result_string = ""

    # Iterate through the stream and accumulate content into the result string
    for chunk in stream:
        result_string += chunk.choices[0].delta.content

    # Extract the feedback label from the response
    feedback = result_string.strip()
    feedback=feedback.lower()
    print(f"this is feedback:{feedback}")
    if "fully relevant" in feedback:
        return "fully relevant"
    if "partially relevant" in feedback:
        return "partially relevant"
    if "irrelevant" in feedback:
        return "irrelevant"
    return "fully relevant"

def adjust_query(query: str, feedback: str, model_name="meta-llama/Meta-Llama-3-8B-Instruct") -> str:
    """
    Adjust the query based on feedback using Llama.

    :param query: The original query.
    :param feedback: Feedback from Llama, such as "Irrelevant", "Partially Relevant", or "Fully Relevant".
    :param model_name: The Llama model name.
    :return: Adjusted query.
    """
    if feedback == "Fully Relevant":
        # No adjustment needed if the feedback is "Fully Relevant"
        return query

    prompt = f"""
    You are an expert query refiner. Your task is to improve or adjust a query based on the feedback provided.
    Feedback categories:
    - "Fully Relevant": The query is perfectly fine and no changes are needed.
    - "Partially Relevant": The query is somewhat clear but could be made more specific or include additional details.
    - "Irrelevant": The query needs significant rephrasing to better match the intended context.

    Feedback: {feedback}
    Original Query: {query}

    Based on the feedback, rewrite the query to improve it.
    """

    client = InferenceClient(api_key="YOUR_API_KEY")

    messages = [{"role": "user", "content": prompt}]

    stream = client.chat.completions.create(
        model=model_name, messages=messages, max_tokens=100, stream=True
    )

    # Initialize an empty string to store the accumulated result
    result_string = ""

    # Iterate through the stream and accumulate content into the result string
    for chunk in stream:
        result_string += chunk.choices[0].delta.content

    # Extract the adjusted query from the response
    adjusted_query = result_string.strip()

    # Ensure the adjusted query is not empty; fallback to original query if needed
    if not adjusted_query:
        adjusted_query = query

    return adjusted_query


In [ ]:
nq_train_df.to_csv('nq_train_df_updated.csv', index=False)


In [ ]:
import pandas as pd
from tqdm import tqdm


def iterative_query_improvement(row, max_iterations=1):
    """
    对单条数据进行查询优化的 RAG 流程。
    :param row: 单条数据的行，包含 query 和 answer
    :param max_iterations: 最大迭代次数
    :return: naive_answer, naive_rag_answer
    """
    query = row['query']
    answer = row['answer']


    key_words = extract_title(query)
    if not key_words:
        return None

    url, title, content = get_wikipedia_page(key_words)
    query_engine = embed_text(content[:10000])

    current_query = query
    for iteration in range(max_iterations):
        query_result = query_engine.query(current_query)
        retrieved_content = query_result.response

        feedback = feedback_evaluate(current_query, retrieved_content)

        if feedback == "fully relevant":
            # improved_rag_answer = llama_answer(question=query, external_knowledge=retrieved_content)
            improved_rag_answer = gpt_answer(question=query, external_knowledge=retrieved_content)

            return improved_rag_answer

        elif feedback == "partially relevant":
            print("改写1")
            current_query = adjust_query(current_query, feedback)

        elif feedback == "irrelevant":
            print("改写2")
            key_words = extract_title(current_query)
            if not key_words:
                return None

            url, title, content = get_wikipedia_page(key_words)
            query_engine = embed_text(content[:20000])

    # improved_rag_answer = llama_answer(question=query, external_knowledge=retrieved_content)
    improved_rag_answer = gpt_answer(question=query, external_knowledge=retrieved_content)

    return improved_rag_answer

# N = 350
# # nq_train_df['improved_rag_answer'] = None

# for index, row in tqdm(nq_train_df.head(N).iterrows(), total=N, desc="Processing rows"):
#     improved_rag_answer = iterative_query_improvement(row)
#     nq_train_df.at[index, 'improved_rag_answer'] = improved_rag_answer
#     print(f"Index {index} completed")
#     print("--------------------------------------------")

# nq_train_df.to_csv('nq_train_df_updated.csv', index=False)


In [ ]:
for index, row in tqdm(nq_train_df.iloc[0:500].iterrows(), total=500, desc="Processing rows"):
    improved_rag_answer = iterative_query_improvement(row)
    nq_train_df.at[index, 'improved_rag_answer'] = improved_rag_answer
    print(f"Index {index} completed")
    print("--------------------------------------------")

nq_train_df.to_csv('gpt_rag.csv', index=False)

Processing rows:   0%|          | 0/500 [00:00<?, ?it/s]

this is feedback:partially relevant
改写1


Processing rows:   0%|          | 1/500 [00:07<1:02:36,  7.53s/it]

Index 0 completed
--------------------------------------------
this is feedback:partially relevant
改写1


Processing rows:   0%|          | 2/500 [00:12<50:08,  6.04s/it]  

...

Index 497 completed
--------------------------------------------
this is feedback:partially relevant
改写1


Processing rows: 100%|█████████▉| 499/500 [51:55<00:04,  4.37s/it]

Index 498 completed
--------------------------------------------
this is feedback:partially relevant
改写1


Processing rows: 100%|██████████| 500/500 [52:00<00:00,  6.24s/it]

Index 499 completed
--------------------------------------------


In [ ]:

for index, row in tqdm(nq_train_df.iloc[500:750].iterrows(), total=250, desc="Processing rows"):
    improved_rag_answer = iterative_query_improvement(row)
    nq_train_df.at[index, 'improved_rag_answer'] = improved_rag_answer
    print(f"Index {index} completed")
    print("--------------------------------------------")

nq_train_df.to_csv('nq_train_df_updated2.csv', index=False)

Processing rows:   0%|          | 0/250 [00:00<?, ?it/s]

this is feedback:fully relevant


Processing rows:   0%|          | 1/250 [00:03<13:04,  3.15s/it]

Index 500 completed
--------------------------------------------
this is feedback:partially relevant
改写1


Processing rows:   1%|          | 2/250 [00:08<19:24,  4.69s/it]

...

Index 747 completed
--------------------------------------------
this is feedback:fully relevant


Processing rows: 100%|█████████▉| 249/250 [22:07<00:03,  3.88s/it]

Index 748 completed
--------------------------------------------
this is feedback:partially relevant
改写1


Processing rows: 100%|██████████| 250/250 [22:13<00:00,  5.34s/it]

Index 749 completed
--------------------------------------------


# Evaluation


In [ ]:
import requests
import numpy as np

# 替换为你的 Hugging Face API 密钥
API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
headers = {"Authorization": "Bearer YOUR_HF_API_KEY"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Hugging Face API error: {response.status_code} - {response.text}")

def calculate_similarity(answer1, answer2):
    """
    计算两个输入句子的相似度。
    :param answer1: 第一个句子
    :param answer2: 第二个句子
    :return: 相似度分数（0到1之间）
    """
    payload = {
        "inputs": {
            "source_sentence": answer1,
            "sentences": [answer2]
        }
    }

    try:
        output = query(payload)
        # Hugging Face 返回的相似度列表
        similarity_score = output[0]
        return similarity_score
    except Exception as e:
        print(f"Error: {e}")
        return None

# 示例调用
if __name__ == "__main__":
    answer1 = "That is a happy person"
    answer2 = "That is a very happy person"

    similarity = calculate_similarity(answer1, answer2)
    if similarity is not None:
        print(f"相似度: {similarity}")

相似度: 0.9429149627685547


In [ ]:
import openai
from openai import OpenAI
import os


client = OpenAI()


def check_answer_equivalence(question, answer1, answer2, model="gpt-4o-mini"):
    """
    判定两个答案是否等价，基于 GPT 模型的输出。

    :param question: 问题
    :param answer1: 第一个答案
    :param answer2: 第二个答案
    :param model: 使用的 GPT 模型，默认为 gpt-4o-mini
    :return: 是否等价（True 或 False）
    """
    prompt = f"""
    Are the following two answers to the given question equivalent? Do not consider whether the answers are right or wrong, but only whether they are equivalent. Directly state \u201dYes\u201d or \u201dNo\u201d.

    Question: {question}
    Answer 1: {answer1}
    Answer 2: {answer2}

    Are the two answers equivalent?"""
    messages = [
        {"role": "system", "content": f"You are an expert at common sense."},
        {"role": "user", "content": prompt},
    ]
    try:
        response = client.chat.completions.create(model=model, messages=messages)
        output = response.choices[0].message.content.strip()
        if output.lower() == "yes":
            return True
        elif output.lower() == "no":
            return False
        else:
            raise ValueError(f"Unexpected model output: {output}")
    except Exception as e:
        print(f"Error: {e}")
        return None


if __name__ == "__main__":
    question = "Which title was conferred to Anna Muzychuk in 2007?"
    answer1 = "Anna Muzychuk was conferred the title of International Master (IM) in 2007. She earned the title by scoring three norms in rapid chess tournaments."
    answer2 = "International Master"

    result = check_answer_equivalence(question, answer1, answer2)
    if result is not None:
        print(f"答案是否等价: {'是' if result else '否'}")


# Download Wikipedia

In [ ]:
import requests
import pandas as pd
from dateutil.parser import parse


def fetch_wikipedia_content(title: str = '2024', verbose: bool = True) -> str:
    """Fetches plaintext content for a given Wikipedia page title using the MediaWiki API."""

    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "exlimit": 1,
        "titles": title,
        "explaintext": True,
        "formatversion": 2,
        "format": "json"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    content = response.json()["query"]["pages"][0]["extract"]

    # Preview content
    if verbose:
        chars = 2500
        print(f"Previewing first {chars} characters of Wikipedia content: \n")
        print(content[:chars], "...\n\n")
    return content


def clean_content(content: str, verbose: bool = True) -> pd.DataFrame:
    """Cleans and processes Wikipedia content into a DataFrame with formatted event descriptions."""

    # Split by newline and create DataFrame
    lines = content.split("\n")
    df = pd.DataFrame(lines, columns=["text"])

    # Remove empty lines and headings (except "Nobel Prizes")
    df = df[~df["text"].str.contains(r"^\s*$")]
    df = df[~df["text"].str.contains("^==.*==$") | df["text"].str.contains("Nobel Prizes")]

    # Display cleaned DataFrame
    if verbose:
        print("Cleaned DataFrame:")
        pd.set_option("display.max_colwidth", 200)
        n_display = 30
        print(df.head(n_display))
        print("\n")
        print(df.tail(n_display))

    return df


def process_events(df: pd.DataFrame, year: str = '2024') -> pd.DataFrame:
    """Processes each line in the DataFrame to format dates and events into a standardized format."""

    processed_df = pd.DataFrame(columns=["text"])
    prefix = f"{year} Overview"

    for _, row in df.iterrows():
        text: str = row["text"]

        # Skip lines that are section headings
        if text.startswith("== "):
            continue

        # Case I: Line is (only) a date or an undated event
        elif " – " not in text[:20]:
            try:
                # Case I.A: Line is a date
                parse(text)
                prefix = text + f', {year}'  # Save date as prefix
            except ValueError:
                # Case I.B: Line is an event (without a date)
                processed_df = add_event(processed_df, prefix, text)

        # Case II: Line is an event with a date or Nobel Prizes field
        else:
            # Extract prefix and attempt parsing as date
            cur_prefix, text_no_prefix = text.split(" – ", 1)
            # Handle cases where the cur_prefix is a range of dates
            cur_prefix_2 = cur_prefix.split("–")[0]
            try:
                # Case II.A: Line is an event with a date
                parse(cur_prefix_2)  # Check if it's a date
                prefix = cur_prefix + f', {year}'  # Save date or duration as prefix
                processed_df = add_event(processed_df, prefix, text_no_prefix)
            except ValueError:
                # Case II.B: Field is likely related to Nobel Prizes
                field_of_study, text = text.split(" – ", 1)
                prefix = f"{year} Nobel Prizes in {field_of_study}"
                processed_df = add_event(processed_df, prefix, text)

    return processed_df


def add_event(df: pd.DataFrame, prefix: str, event_text: str) -> pd.DataFrame:
    """Appends a new row with a formatted event to the DataFrame."""

    new_row = {"text": f"{prefix} – {event_text}"}
    concat_df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    return concat_df


def fetch_and_process_wikipedia_content(title: str, verbose: bool = True) -> pd.DataFrame:
    """Fetches and processes Wikipedia content into a DataFrame with formatted event descriptions."""

    # Fetch Wikipedia content and clean it
    content = fetch_wikipedia_content(title, verbose)
    df = clean_content(content, verbose)
    processed_df = process_events(df, title)

    # Display processed DataFrame
    if verbose:
        print("Processed DataFrame:")
        pd.set_option("display.max_colwidth", 200)
        n_display = 60
        print(processed_df.head(n_display))
        print("\n")
        print(processed_df.tail(n_display))

    return processed_df

In [ ]:
user_question = "巴黎的著名景点"
generated_title = "Tourist attractions in Paris"  # 或通过模型生成


In [ ]:
title = generated_title  # 模型生成的标题
processed_df = fetch_and_process_wikipedia_content(title)


KeyError: 'extract'

In [ ]:
!pip install blingfire


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 50.1 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import time
import re
import gzip
import gc    # Garbage collector
from tqdm.auto import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed

import blingfire as bf
import numpy as np
from datasets import load_dataset


In [ ]:
# Compile regular expressions only once at the beginning
infobox_pattern = re.compile(r'\{\{Infobox [^}]+\}\}', flags=re.DOTALL)
sidebar_pattern = re.compile(r'\{\{Sidebar [^}]+\}\}', flags=re.DOTALL)
link_pattern = re.compile(r'\[\[([^|\]]+\|)?([^\]]+)\]\]')
references_pattern = re.compile(r'==\s*(References|External links|See also|Notes)\s*==.*', flags=re.DOTALL)
citation_needed_pattern = re.compile(r'\{\{citation needed[^}]*\}\}', flags=re.DOTALL)
cn_pattern = re.compile(r'\{\{cn\}\}', flags=re.DOTALL)
curly_braces_pattern = re.compile(r'\{\{[^}]+\}\}', flags=re.DOTALL)
whitespace_pattern = re.compile(r'\s+')


In [ ]:
def preprocess_article(text: str) -> str:
    # Remove infoboxes and sidebars
    text = infobox_pattern.sub('', text)
    text = sidebar_pattern.sub('', text)

    # Simplify links - keep the text of the link only
    text = link_pattern.sub(r'\2', text)

    # Remove sections that start with == References ==, == External links ==, etc.
    text = references_pattern.sub('', text)

    # Optional: Remove citation needed and similar templates
    text = citation_needed_pattern.sub('', text)
    text = cn_pattern.sub('', text)  # Short form of citation needed

    # Remove any remaining curly braces content (catch-all for other templates)
    text = curly_braces_pattern.sub('', text)

    # Normalize whitespace to a single space
    text = whitespace_pattern.sub(' ', text).strip()

    return text

In [ ]:
def process_article(article_text: str, min_len: int, max_len: int) -> str:
    # Preprocess the article text with regex
    article_text = preprocess_article(article_text)

    # Skip the processing if there is no relavant sentences
    if not article_text:
        return ""

    # Initialize an empty list to hold sentences that meet the length criteria.
    proper_sentences = []

    # Segment the preprocessed article text into sentences and obtain their offsets (start and end positions).
    _, offsets = bf.text_to_sentences_and_offsets(article_text)

    for o in offsets:
        # Check if the length of the current sentence (calculated as end position - start position)
        # falls within the specified minimum and maximum length bounds.
        if not min_len <= o[1] - o[0] <= max_len:
            # If the sentence does not meet the length criteria, skip to the next iteration.
            continue
        sentence = article_text[o[0]:o[1]]

        # Add the sentence that meets the length criteria to the list of proper sentences.
        proper_sentences.append(sentence)

    # Join the proper sentences into a single string, separated by newline characters,
    # and return this string. This results in a string where each sentence is on a new line,
    # assuming it met the length criteria.
    return '\n'.join(proper_sentences)

def process_article_wrapper(args):
    return process_article(*args)

In [ ]:
def preprocess_article(text: str) -> str:
    # Remove infoboxes and sidebars
    text = infobox_pattern.sub('', text)
    text = sidebar_pattern.sub('', text)

    # Simplify links - keep the text of the link only
    text = link_pattern.sub(r'\2', text)

    # Remove sections that start with == References ==, == External links ==, etc.
    text = references_pattern.sub('', text)

    # Optional: Remove citation needed and similar templates
    text = citation_needed_pattern.sub('', text)
    text = cn_pattern.sub('', text)  # Short form of citation needed

    # Remove any remaining curly braces content (catch-all for other templates)
    text = curly_braces_pattern.sub('', text)

    # Normalize whitespace to a single space
    text = whitespace_pattern.sub(' ', text).strip()

    return text

def process_article(article_text: str, min_len: int, max_len: int) -> str:
    # Preprocess the article text with regex
    article_text = preprocess_article(article_text)

    # Skip the processing if there is no relavant sentences
    if not article_text:
        return ""

    # Initialize an empty list to hold sentences that meet the length criteria.
    proper_sentences = []

    # Segment the preprocessed article text into sentences and obtain their offsets (start and end positions).
    _, offsets = bf.text_to_sentences_and_offsets(article_text)

    for o in offsets:
        # Check if the length of the current sentence (calculated as end position - start position)
        # falls within the specified minimum and maximum length bounds.
        if not min_len <= o[1] - o[0] <= max_len:
            # If the sentence does not meet the length criteria, skip to the next iteration.
            continue

        # Extract the sentence from the article text using the start and end positions from 'o'.
        sentence = article_text[o[0]:o[1]]

        # Add the sentence that meets the length criteria to the list of proper sentences.
        proper_sentences.append(sentence)

    # Join the proper sentences into a single string, separated by newline characters,
    # and return this string. This results in a string where each sentence is on a new line,
    # assuming it met the length criteria.
    return '\n'.join(proper_sentences)

def process_article_wrapper(args):
    return process_article(*args)

def process_wikipedia_dataset(wiki_dataset, output_dir, articles_per_file=1_000_000, batch_size=100):
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    file_count = 1
    article_count = 0
    out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')

    with ProcessPoolExecutor() as executor:
        futures = {}
        for article in tqdm(wiki_dataset, desc='Processing Articles'):
            # Submit tasks as you iterate
            future = executor.submit(process_article_wrapper, (article['text'], 32, 2048))
            futures[future] = article['text']

            # Process completed tasks in batches to save memory
            if len(futures) >= batch_size:
                for future in as_completed(futures):
                    sentences = future.result()
                    out_f.write(sentences + '\n')
                    article_count += 1

                    if article_count >= articles_per_file:
                        out_f.close()
                        file_count += 1
                        article_count = 0
                        out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')
                    # Remove the future from the dictionary once processed
                    del futures[future]
                    break  # Break after processing one to check if more tasks should be added

        # Process any remaining tasks
        for future in as_completed(futures):
            sentences = future.result()
            out_f.write(sentences + '\n')
            article_count += 1

            if article_count >= articles_per_file:
                out_f.close()
                file_count += 1
                article_count = 0
                out_f = gzip.open(f'{output_dir}/wikipedia_processed_{file_count}.txt.gz', 'wt', encoding='utf-8')

    # Ensure the last file is closed
    out_f.close()

In [ ]:
%pwd

'/content'

In [ ]:
# Load the Wikipedia dataset
wiki_dataset = load_dataset("wikipedia", "20220301.en", split='train')
print(f'Length of the Wikipedia dataset is {len(wiki_dataset):_} articles.')

# Specify the working directory to save the processed files
output_dir = '/content/cse595'
# Process the Wikipedia dataset add them to .txt.gz files
process_wikipedia_dataset(wiki_dataset, output_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00041.parquet:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

...

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

Length of the Wikipedia dataset is 6_458_670 articles.


Processing Articles:   0%|          | 0/6458670 [00:00<?, ?it/s]

Process ForkProcess-7:
Process ForkProcess-9:
Process ForkProcess-11:
Process ForkProcess-6:
Process ForkProcess-12:
Process ForkProcess-3:
Process ForkProcess-2:
Process ForkProcess-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py

KeyboardInterrupt: 